I have two codes for motion detection. Test both

In [12]:
from imutils.video import VideoStream
import argparse
import datetime
import imutils
import time
import cv2
import json
import warnings
import numpy as np

In [40]:
def init_data(point):
    data = {}
    data["points"] = [point]
    data["offsets"] = []
    data["last_slope"] = None
    return data

def same_point(point1, point2):
    if point1 == point2:
        return True
    if point1[0] == point2[0]:
        return None  # corresponds to infinite slope
    else:
        return False

In [ ]:
def get_slope(point1, point2):
    

In [120]:
def is_straight_line(point, data, R=50):
    """
    Returns true if the third point is within the "band" defiend by previous points
    """
    list_of_previous_points = data["points"]
    offsets = data["offsets"]
    last_slope = data["last_slope"]
    last_point = list_of_previous_points[-1]
    initial_point = list_of_previous_points[0]

    if len(list_of_previous_points) >= 2:
        if same_point(point, last_point) is False:
            if (point[0] - last_point[0])**2 + (point[1] - last_point[1])**2 <= R**2:  # point is in close proximity
                slope_to_test = (point[1]-last_point[1])/(point[0]-last_point[0])
                slope_init = (point[1]-initial_point[1])/(point[0]-initial_point[0])
                offset = point[1] - slope_to_test*point[0]
                if len(offsets) >=1:
                    offset_average = sum(offsets)/len(offsets)
                else:
                    offset_average = offset
                if check_constraints(slope_init, last_slope, slope_to_test, offset, offset_average) is True:
                    print("passed constraints")
                    list_of_previous_points.append(point)
                    offsets.append(offset)
                    # update dict
                    data["points"] = list_of_previous_points
                    data["offsets"] = offsets
                    data["last_slope"] = slope_to_test
                if check_constraints(slope_init, last_slope, slope_to_test, offset, offset_average) is False:
                    print("failed at constraints")

        if same_point(point, last_point) is True:
            list_of_previous_points.append(point)
            data["points"] = list_of_previous_points

    if len(list_of_previous_points) == 1:
        if same_point(point, initial_point) is False:
            slope = (point[1]-initial_point[1])/(point[0]-initial_point[0])
            offset = point[1] - slope*point[0]
            offsets.append(offset)
            # update dict
            data["offsets"] = offsets
            data["last_slope"] = slope
            
        list_of_previous_points.append(point)
        data["points"] = list_of_previous_points
    
    return data
            
        
def check_slope(slope1, slope2, slope3, offset_avg, slope_error=1):
    """
    checks if slope3 is similar to slopes 2 and 3.
    checks if offset is close to the average
    
    """
    if slope2 is None:
        if abs(slope1 - slope3) <= slope_error:
            return True
    if slope2 is not None:
        if abs(slope1 - slope3) <= slope_error and abs(slope2 - slope3) <= slope_error:
            return True
        else:
            return False
    else:
        return False

In [29]:
def check_for_lines(datas,point):
    if bool(datas) is False:  # no data points yet
        data = init_data(point)
        datas.append(data)
    else:
        for i in range(len(datas)):
            line = is_straight_line(point,datas[i])
            datas[i] = line
    return datas

In [121]:
video_location = 'Earth_ball.avi'
vs = cv2.VideoCapture(video_location)

In [122]:
# initialize the first frame in the video stream
avg = None
# initialise data on debris lines
datas = []

In [123]:
#main body
accumulated_thresh = np.zeros((1080,1920))
while True:
    frame = vs.read()
    frame = frame[1]
    text = "unoccupied"
    
    # if the frame could not be grabbed, then we have reached the end
    # of the video
    if frame is None:
        break
    
    # resize the frame, convert it to grayscale, and blur it
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)

    # if the first frame is None, initialize it
    if avg is None:
        avg = gray.copy().astype("float")
        continue

    # compute the absolute difference between the current frame and
    # first frame
    cv2.accumulateWeighted(gray, avg, 0.5)
    frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(avg))
    thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1]

    # dilate the thresholded image to fill in holes, then find contours
    # on thresholded image
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_TREE,
    	cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    # save the contours into an accumulated thresh image
    accumulated_thresh = cv2.accumulate(thresh, accumulated_thresh)

	# loop over the contours
    for c in cnts:
        if cv2.contourArea(c) > 200:
		# compute the bounding box for the contour, draw it on the frame,
		# and update the text
            """
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            """
            ellipse = cv2.fitEllipse(c)
            cv2.ellipse(frame,ellipse,(0,255,0),2)
            text = "Occupied"
            # get the location of the centroid
            M = cv2.moments(c)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            print([cx,cy])
            datas = check_for_lines(datas, [cx,cy])
            print("datas", datas)

        

    # draw the text and timestamp on the frame
    #cv2.putText(frame, "Room Status: {}".format(text), (10, 20),
    #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    #cv2.putText(frame, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"),
    #(10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)

	# show the frame and record if the user presses a key
    """
    cv2.imshow("Satellite Feed", frame)
    cv2.imshow("accumulated", accumulated_thresh)
    cv2.imshow("Thresh", thresh)
    cv2.imshow("Frame Delta", frameDelta)
    cv2.waitKey(0)
    """
    
cv2.destroyAllWindows()

[994, 539]
datas [{'points': [[994, 539]], 'offsets': [], 'last_slope': None}]
[994, 539]
datas [{'points': [[994, 539], [994, 539]], 'offsets': [], 'last_slope': None}]
[999, 539]
passed constraints
datas [{'points': [[994, 539], [994, 539], [999, 539]], 'offsets': [539.0], 'last_slope': 0.0}]
[1009, 538]
passed constraints
datas [{'points': [[994, 539], [994, 539], [999, 539], [1009, 538]], 'offsets': [539.0, 638.9], 'last_slope': -0.1}]
[1018, 537]
passed constraints
datas [{'points': [[994, 539], [994, 539], [999, 539], [1009, 538], [1018, 537]], 'offsets': [539.0, 638.9, 650.1111111111111], 'last_slope': -0.1111111111111111}]
[1027, 536]
passed constraints
datas [{'points': [[994, 539], [994, 539], [999, 539], [1009, 538], [1018, 537], [1027, 536]], 'offsets': [539.0, 638.9, 650.1111111111111, 650.1111111111111], 'last_slope': -0.1111111111111111}]
[1036, 535]
passed constraints
datas [{'points': [[994, 539], [994, 539], [999, 539], [1009, 538], [1018, 537], [1027, 536], [1036, 53

#Method 2

In [3]:
# KNN
backSub = cv2.createBackgroundSubtractorKNN()

In [14]:
# MOG2
backSub = cv2.createBackgroundSubtractorMOG2()

In [4]:
while True:
    ret, frame = vs.read()
    if frame is None:
        break

    fgMask = backSub.apply(frame)

    thresh = cv2.dilate(fgMask, None, iterations=2)  # makes bright regions grow
    cnts = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # loop over the contours
    for c in cnts:
        if cv2.contourArea(c) > 200:
            # compute the bounding box for the contour, draw it on the frame
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            text = "Occupied"

    cv2.rectangle(frame, (10, 2), (100, 20), (255, 255, 255), 1)
   
    cv2.imshow('Frame', frame)
    cv2.imshow('Thresh', thresh)
    cv2.imshow('FG Mask', fgMask)

    cv2.waitKey(0)
    
cv2.destroyAllWindows()

KeyboardInterrupt: 